In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

loaded_sentence_embeddings = torch.load("final_sentence_embeddings.pt")
loaded_sequence_embeddings = torch.load("final_sequence_embeddings.pt")

shot_count=torch.load('shot_count.pt')
edit_trends=torch.load('edit_trends_labels.pt')


In [8]:
n_shot_class=[]
for i in range(len(shot_count)):
    
    if shot_count[i]<=20:
        n_shot_class.append(0)
    elif shot_count[i]>20 and shot_count[i]<=40:
        n_shot_class.append(1)
    elif shot_count[i]>40 and shot_count[i]<=60:
        n_shot_class.append(2)
    elif shot_count[i]>60:
        n_shot_class.append(3)
    



In [9]:
shot_labs=torch.tensor(n_shot_class)
n_shot_class_1hot=torch.zeros(shot_labs.size(0), 4)
n_shot_class_1hot.scatter_(1, shot_labs.unsqueeze(1), 1)


tensor([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]])

In [10]:
edit_trends_lab=torch.tensor(edit_trends)
edit_trends_1hot=torch.zeros(edit_trends_lab.size(0), 5)
edit_trends_1hot.scatter_(1, edit_trends_lab.unsqueeze(1), 1)

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        ...,
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split

#version with editing mix/trend and length label

# Define the Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        return x
    
# Define the Decoder
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

# Define the Seq2Seq Model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg, label, emix):
        # Concatenate label with src
        src = torch.cat((src, label), dim=1)
        src = torch.cat((src, emix), dim=1)
        src = self.encoder(src)
        output = self.decoder(src)
        return output

# Define the Cosine Similarity Loss
class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()

    def forward(self, output, target):
        
        output_normalized = F.normalize(output, p=2, dim=-1)
        target_normalized = F.normalize(target, p=2, dim=-1)
        
        # Compute cosine similarity
        cosine_sim = F.cosine_similarity(output_normalized, target_normalized, dim=-1)
        
        
        loss = 1 - cosine_sim  
        
        return loss.mean()

dataset1 = loaded_sentence_embeddings  
dataset2 = loaded_sequence_embeddings  

len_lab=torch.tensor(n_shot_class_1hot, dtype=torch.float)
trend_lab=torch.tensor(edit_trends_1hot, dtype=torch.float)
dataset = TensorDataset(torch.FloatTensor(dataset1), torch.FloatTensor(dataset2), torch.FloatTensor(len_lab), torch.FloatTensor(trend_lab))
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


# Define the split ratio (e.g. 80% for training and 20% for testing)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split dataset into training and testing datasets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for both train and test datasets
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)




# Initialize models
input_size = 768  
hidden_size = 512  
hidden_size_2 = 384  
output_size = 256  

#encoder = Encoder(input_size , hidden_size)
encoder = Encoder(input_size + len(len_lab[0])+ len(trend_lab[0]), hidden_size)  # Adding label size to input_size
decoder = Decoder(hidden_size, hidden_size_2, output_size)#working for 256 configuration

model = Seq2Seq(encoder, decoder)

# Loss and optimizer
cosine_loss = CosineSimilarityLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 200

# Training loop
for epoch in range(epochs):
    print(epoch)
    model.train()  # Set model to training mode
    running_loss = 0.0
    cosine_sim_accumulator = 0.0
    num_samples = 0
    for i, (input_data, target_data, label_data, trend_data) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        # Forward pass
        output = model(input_data, target_data, label_data, trend_data)
        
        # Compute the cosine similarity loss
        loss = cosine_loss(output, target_data)
        running_loss += loss.item()
        
        # Compute cosine similarity
        output_normalized = F.normalize(output, p=2, dim=-1)
        target_normalized = F.normalize(target_data, p=2, dim=-1)
        cosine_sim = F.cosine_similarity(output_normalized, target_normalized, dim=-1)
        cosine_sim_accumulator += cosine_sim.sum().item()
        num_samples += output.size(0)
        
        # Backward pass
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        avg_loss = running_loss / len(train_dataloader)
        avg_cosine_sim = cosine_sim_accumulator / num_samples
        print(f'Training - Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Avg Cosine Similarity: {avg_cosine_sim:.4f}')

# Validation loop
model.eval()  
with torch.no_grad():
    val_running_loss = 0.0
    val_cosine_sim_accumulator = 0.0
    val_num_samples = 0
    for input_data_val, target_data_val, label_data_val, trend_data_val in test_dataloader:
        val_output = model(input_data_val, target_data_val, label_data_val, trend_data_val)
        
        # Compute the cosine similarity loss
        val_loss = cosine_loss(val_output, target_data_val)
        val_running_loss += val_loss.item()
        
        # Compute cosine similarity
        val_output_normalized = F.normalize(val_output, p=2, dim=-1)
        target_normalized_val = F.normalize(target_data_val, p=2, dim=-1)
        val_cosine_sim = F.cosine_similarity(val_output_normalized, target_normalized_val, dim=-1)
        val_cosine_sim_accumulator += val_cosine_sim.sum().item()
        val_num_samples += val_output.size(0)
    
    avg_val_loss = val_running_loss / len(test_dataloader)
    avg_val_cosine_sim = val_cosine_sim_accumulator / val_num_samples
    print(f'Validation Loss: {avg_val_loss:.4f}, Avg Cosine Similarity: {avg_val_cosine_sim:.4f}')


/tmp/ipykernel_567/510770044.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  len_lab=torch.tensor(n_shot_class_1hot, dtype=torch.float)
/tmp/ipykernel_567/510770044.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trend_lab=torch.tensor(edit_trends_1hot, dtype=torch.float)


0


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split

#version with only length label

# Define the Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        return x
    
# Define the Decoder
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

# Define the Seq2Seq Model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg, label):
        # Concatenate label with src
        src = torch.cat((src, label), dim=1)
        src = self.encoder(src)
        output = self.decoder(src)
        return output
#"""
# Define the Cosine Similarity Loss
class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()

    def forward(self, output, target):
        # Normalize the embeddings
        output_normalized = F.normalize(output, p=2, dim=-1)
        target_normalized = F.normalize(target, p=2, dim=-1)
        
        # Compute cosine similarity
        cosine_sim = F.cosine_similarity(output_normalized, target_normalized, dim=-1)
        
        # Cosine similarity loss
        loss = 1 - cosine_sim  # minimize the angle, maximize cosine similarity
        
        return loss.mean()

    
dataset1 = loaded_sentence_embeddings  
dataset2 = loaded_sequence_embeddings  

len_lab=torch.tensor(n_shot_class_1hot, dtype=torch.float)
dataset = TensorDataset(torch.FloatTensor(dataset1), torch.FloatTensor(dataset2), torch.FloatTensor(len_lab))
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


# Define the split ratio (e.g. 80% for training and 20% for testing)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split dataset into training and testing datasets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for both train and test datasets
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)




# Initialize models
input_size = 768  
hidden_size = 512  
hidden_size_2 = 384  
output_size = 256  

#encoder = Encoder(input_size , hidden_size)
encoder = Encoder(input_size + len(len_lab[0]), hidden_size)  # Adding label size to input_size
decoder = Decoder(hidden_size, hidden_size_2, output_size)#working for 256 configuration

model = Seq2Seq(encoder, decoder)

# Loss and optimizer
cosine_loss = CosineSimilarityLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 200

# Training loop
for epoch in range(epochs):
    print(epoch)
    model.train()  # Set model to training mode
    running_loss = 0.0
    cosine_sim_accumulator = 0.0
    num_samples = 0
    for i, (input_data, target_data, label_data) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        # Forward pass
        output = model(input_data, target_data, label_data)
        
        # Compute the cosine similarity loss
        loss = cosine_loss(output, target_data)
        running_loss += loss.item()
        
        # Compute cosine similarity
        output_normalized = F.normalize(output, p=2, dim=-1)
        target_normalized = F.normalize(target_data, p=2, dim=-1)
        cosine_sim = F.cosine_similarity(output_normalized, target_normalized, dim=-1)
        cosine_sim_accumulator += cosine_sim.sum().item()
        num_samples += output.size(0)
        
        # Backward pass
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        avg_loss = running_loss / len(train_dataloader)
        avg_cosine_sim = cosine_sim_accumulator / num_samples
        print(f'Training - Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Avg Cosine Similarity: {avg_cosine_sim:.4f}')

# Validation loop
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    val_running_loss = 0.0
    val_cosine_sim_accumulator = 0.0
    val_num_samples = 0
    for input_data_val, target_data_val, label_data_val in test_dataloader:
        val_output = model(input_data_val, target_data_val, label_data_val)
        
        # Compute the cosine similarity loss
        val_loss = cosine_loss(val_output, target_data_val)
        val_running_loss += val_loss.item()
        
        # Compute cosine similarity
        val_output_normalized = F.normalize(val_output, p=2, dim=-1)
        target_normalized_val = F.normalize(target_data_val, p=2, dim=-1)
        val_cosine_sim = F.cosine_similarity(val_output_normalized, target_normalized_val, dim=-1)
        val_cosine_sim_accumulator += val_cosine_sim.sum().item()
        val_num_samples += val_output.size(0)
    
    avg_val_loss = val_running_loss / len(test_dataloader)
    avg_val_cosine_sim = val_cosine_sim_accumulator / val_num_samples
    print(f'Validation Loss: {avg_val_loss:.4f}, Avg Cosine Similarity: {avg_val_cosine_sim:.4f}')


/tmp/ipykernel_180/894204831.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(one_hot_labels, dtype=torch.float)
/tmp/ipykernel_180/894204831.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  len_lab=torch.tensor(n_shot_class_1hot, dtype=torch.float)


0
1
2
3
4
5
6
7
8
9
Training - Epoch [10/200], Loss: 0.0984, Avg Cosine Similarity: 0.9016
10
11
12
13
14
15
16
17
18
19
Training - Epoch [20/200], Loss: 0.0930, Avg Cosine Similarity: 0.9070
20
21
22
23
24
25
26
27
28
29
Training - Epoch [30/200], Loss: 0.0871, Avg Cosine Similarity: 0.9129
30
31
32
33
34
35
36
37
38
39
Training - Epoch [40/200], Loss: 0.0822, Avg Cosine Similarity: 0.9178
40
41
42
43
44
45
46
47
48
49
Training - Epoch [50/200], Loss: 0.0787, Avg Cosine Similarity: 0.9213
50
51
52
53
54
55
56
57
58
59
Training - Epoch [60/200], Loss: 0.0762, Avg Cosine Similarity: 0.9238
60
61
62
63
64
65
66
67
68
69
Training - Epoch [70/200], Loss: 0.0743, Avg Cosine Similarity: 0.9257
70
71
72
73
74
75
76
77
78
79
Training - Epoch [80/200], Loss: 0.0727, Avg Cosine Similarity: 0.9273
80
81
82
83
84
85
86
87
88
89
Training - Epoch [90/200], Loss: 0.0714, Avg Cosine Similarity: 0.9286
90
91
92
93
94
95
96
97
98
99
Training - Epoch [100/200], Loss: 0.0703, Avg Cosine Similarity: 0.9297